# Szakdolgozat


## Fine-tuning

### Installok, importok, huggingface login

In [ ]:
# Installok
!pip install transformers[torch] datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00


In [ ]:
# Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Importok
import string
import re
import evaluate
import spacy
import numpy as np

from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

nlp_en = spacy.load('en_core_web_sm')

### Dataset betöltés, előfeldolgozás

In [ ]:
# Dataset betöltés
dataset = load_dataset('nanyy1025/covid_fake_news')
print(dataset)

# 'id' oszlop törlése
dataset = dataset.remove_columns(['id'])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tweet', 'label'],
        num_rows: 6420
    })
    validation: Dataset({
        features: ['id', 'tweet', 'label'],
        num_rows: 2140
    })
    test: Dataset({
        features: ['id', 'tweet', 'label'],
        num_rows: 2140
    })
})


In [ ]:
# Printelés előfeldolgozás előtt
print('Before preprocessing:')
print(dataset['train'][89])

Before preprocessing:
{'tweet': '📢#CoronaVirusUpdates: 📍#COVID19 India Tracker (As on 7 September 2020 08:00 AM) ➡️Confirmed cases: 4204613 ➡️Recovered: 3250429 (77.3%)👍 ➡️Active cases: 882542 (21.0%) ➡️Deaths: 71642 (1.7%) #IndiaFightsCorona #IndiaWillWin #StaySafe Via @MoHFW_INDIA https://t.co/WVQslxqBPC', 'label': 'real'}


In [ ]:
# Előfeldolgozás
def data_preprocess(data):
    # Label "fordítás"
    if data['label'] == 'real':
      data['label'] = 1
    elif data['label'] == 'fake':
      data['label'] = 0

    # Kisbetűsítés
    #data['tweet'] = data['tweet'].lower()

    # Stopwordök törlése
    #doc = nlp_en(data['tweet'])
    #data['tweet'] = [token.text for token in doc if not token.is_stop]
    #data['tweet'] = ' '.join(data['tweet'])

    # URLek törlése
    #url_pattern = re.compile(r'https?://\S+|www\.\S+')
    #data['tweet'] = url_pattern.sub(r'', data['tweet'])

    # HTML speciális karakterek törlése
    #special_pattern = re.compile(r'&.*?;')
    #data['tweet'] = special_pattern.sub(r'', data['tweet'])

    # Emojik törlése
    #emoji_pattern = re.compile('['
    #                       u'\U0001F600-\U0001F64F'  # emotikonok
    #                       u'\U0001F300-\U0001F5FF'  # szimbólumok és piktogramok
    #                       u'\U0001F680-\U0001F6FF'  # közlekedés és térkép szimbólumok
    #                       u'\U0001F1E0-\U0001F1FF'  # zászlók (iOS)
    #                       u'\U00002702-\U000027B0'
    #                       u'\U000024C2-\U0001F251'
    #                       ']+', flags=re.UNICODE)
    #data['tweet'] = emoji_pattern.sub(r'', data['tweet'])

    # Írásjelek törlése
    #punct_list = string.punctuation
    #punct_table = str.maketrans(dict.fromkeys(punct_list, ' '))
    #data['tweet'] = data['tweet'].translate(punct_table)

    # Felesleges whitespaceek törlése, splitelés és joinolás
    data['tweet'] = data['tweet'].strip().split()
    data['tweet'] = ' '.join(data['tweet'])

    return data


dataset_processed = dataset.map(data_preprocess)

Map:   0%|          | 0/6420 [00:00<?, ? examples/s]

Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

In [ ]:
# Printelés előfeldolgozás után
print('After preprocessing:')
print(dataset_processed['train'][89])

After preprocessing:
{'tweet': '📢#CoronaVirusUpdates: 📍#COVID19 India Tracker (As on 7 September 2020 08:00 AM) ➡️Confirmed cases: 4204613 ➡️Recovered: 3250429 (77.3%)👍 ➡️Active cases: 882542 (21.0%) ➡️Deaths: 71642 (1.7%) #IndiaFightsCorona #IndiaWillWin #StaySafe Via @MoHFW_INDIA https://t.co/WVQslxqBPC', 'label': 1}


### Tokenizálás, kiértékelő függvény

In [ ]:
# Tokenizálás
model_name = 'microsoft/deberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenizer_function(data):
    return tokenizer(data['tweet'], truncation=True)


dataset_tokenized = dataset_processed.map(tokenizer_function, batched=True)

Map:   0%|          | 0/6420 [00:00<?, ? examples/s]

Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

In [ ]:
# Printelés tokenizálás után
print('After tokenizing:')
print(dataset_tokenized['train'][89])

After tokenizing:
{'tweet': '📢#CoronaVirusUpdates: 📍#COVID19 India Tracker (As on 7 September 2020 08:00 AM) ➡️Confirmed cases: 4204613 ➡️Recovered: 3250429 (77.3%)👍 ➡️Active cases: 882542 (21.0%) ➡️Deaths: 71642 (1.7%) #IndiaFightsCorona #IndiaWillWin #StaySafe Via @MoHFW_INDIA https://t.co/WVQslxqBPC', 'label': 1, 'input_ids': [1, 6569, 9085, 7258, 10431, 15228, 4488, 846, 19473, 10926, 19718, 35, 8103, 9085, 8384, 10431, 6335, 43814, 1646, 666, 35368, 36, 1620, 15, 262, 772, 2760, 12112, 35, 612, 3326, 43, 14333, 17772, 5543, 12605, 33295, 9125, 1200, 35, 26748, 3761, 1558, 14333, 17772, 5543, 12605, 21109, 16223, 35, 2107, 1096, 31877, 36, 4718, 4, 246, 8871, 31193, 8384, 14333, 17772, 5543, 12605, 42586, 1200, 35, 7953, 1244, 3714, 36, 2146, 4, 288, 8871, 14333, 17772, 5543, 12605, 34811, 29, 35, 262, 1549, 3714, 36, 134, 4, 406, 8871, 849, 11015, 597, 6183, 15228, 4488, 849, 11015, 17485, 27189, 849, 34112, 32356, 17442, 787, 17357, 725, 18514, 1215, 13796, 2889, 1205, 640, 90, 4

In [ ]:
# Kiértékelő, id2label, label2id
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)


id2label = {0: 'FAKE', 1: 'REAL'}
label2id = {'FAKE': 0, 'REAL': 1}

### Tanítás beállítások

In [ ]:
# Model, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

output_dir = 'FakeNews-deberta-large-stable'

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    adam_epsilon=1e-6,
    adam_beta1=0.9,
    adam_beta2=0.999,
    weight_decay=0.01,
    max_grad_norm=1.0,
    load_best_model_at_end=True,
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenized['train'],
    eval_dataset=dataset_tokenized['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Fine-tuning eredmények

#### Bert-base-cased

##### Nincs adat előfeldolgozás

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.280100,0.164430,0.966355
2,0.069400,0.094107,0.977570
3,0.029300,0.135190,0.979907
4,0.014000,0.123272,0.981776
5,0.003600,0.135675,0.979439


TrainOutput(global_step=4015, training_loss=0.0690236829325776, metrics={'train_runtime': 770.2967, 'train_samples_per_second': 41.672, 'train_steps_per_second': 5.212, 'total_flos': 1642355651998080.0, 'train_loss': 0.0690236829325776, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.0941069945693016,
 'eval_accuracy': 0.9775700934579439,
 'eval_runtime': 12.7487,
 'eval_samples_per_second': 167.861,
 'eval_steps_per_second': 21.022,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-bert-base-cased/tree/main/'

##### + Kisbetűsítés

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.253900,0.208555,0.953738
2,0.078800,0.190636,0.964486
3,0.030400,0.271643,0.962617
4,0.009400,0.162848,0.974766
5,0.003200,0.176614,0.974299


TrainOutput(global_step=4015, training_loss=0.06836164521098666, metrics={'train_runtime': 749.7661, 'train_samples_per_second': 42.813, 'train_steps_per_second': 5.355, 'total_flos': 1590210329713920.0, 'train_loss': 0.06836164521098666, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.1628478467464447,
 'eval_accuracy': 0.974766355140187,
 'eval_runtime': 12.4034,
 'eval_samples_per_second': 172.534,
 'eval_steps_per_second': 21.607,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-bert-base-cased-lowercase/tree/main/'

##### + Stopwordök törlése

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.302300,0.268179,0.936916
2,0.086100,0.247322,0.953738
3,0.042900,0.169368,0.966355
4,0.012400,0.171478,0.972897
5,0.004500,0.191477,0.971495


TrainOutput(global_step=4015, training_loss=0.08282583149313547, metrics={'train_runtime': 687.0633, 'train_samples_per_second': 46.721, 'train_steps_per_second': 5.844, 'total_flos': 1387053650538960.0, 'train_loss': 0.08282583149313547, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.16936784982681274,
 'eval_accuracy': 0.9663551401869159,
 'eval_runtime': 11.3421,
 'eval_samples_per_second': 188.678,
 'eval_steps_per_second': 23.629,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-bert-base-cased-stopwords/tree/main/'

##### + URLek törlése

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.323200,0.195737,0.946262
2,0.127000,0.170748,0.956542
3,0.062100,0.232171,0.959813
4,0.017600,0.338857,0.952336
5,0.005600,0.269755,0.963084


TrainOutput(global_step=4015, training_loss=0.09840696567111645, metrics={'train_runtime': 602.3982, 'train_samples_per_second': 53.287, 'train_steps_per_second': 6.665, 'total_flos': 1094994212424000.0, 'train_loss': 0.09840696567111645, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.17074823379516602,
 'eval_accuracy': 0.9565420560747664,
 'eval_runtime': 9.2979,
 'eval_samples_per_second': 230.16,
 'eval_steps_per_second': 28.824,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-bert-base-cased-url/tree/main/'

##### + HTML speciális karakterek törlése

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.343500,0.286327,0.941589
2,0.112100,0.290389,0.939720
3,0.060100,0.238187,0.962150
4,0.014600,0.229085,0.962150
5,0.007800,0.255350,0.965421


TrainOutput(global_step=4015, training_loss=0.09826104853854307, metrics={'train_runtime': 599.8411, 'train_samples_per_second': 53.514, 'train_steps_per_second': 6.693, 'total_flos': 1081221993120000.0, 'train_loss': 0.09826104853854307, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.22908549010753632,
 'eval_accuracy': 0.9621495327102804,
 'eval_runtime': 9.2946,
 'eval_samples_per_second': 230.241,
 'eval_steps_per_second': 28.834,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-bert-base-cased-html/tree/main/'

##### + Emojik törlése

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.327200,0.328922,0.934579
2,0.130400,0.191270,0.957009
3,0.062800,0.250684,0.959346
4,0.034200,0.230632,0.959813
5,0.010400,0.226536,0.965888


TrainOutput(global_step=4015, training_loss=0.10452114364392082, metrics={'train_runtime': 612.9118, 'train_samples_per_second': 52.373, 'train_steps_per_second': 6.551, 'total_flos': 1071523884063840.0, 'train_loss': 0.10452114364392082, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.19126977026462555,
 'eval_accuracy': 0.9570093457943926,
 'eval_runtime': 9.0073,
 'eval_samples_per_second': 237.585,
 'eval_steps_per_second': 29.754,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-bert-base-cased-emoji/tree/main/'

##### + Írásjelek törlése

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.373400,0.205777,0.931776
2,0.173200,0.237169,0.940654
3,0.091400,0.325096,0.937850
4,0.039100,0.402220,0.938318
5,0.017100,0.414453,0.941589


TrainOutput(global_step=4015, training_loss=0.12738031746058784, metrics={'train_runtime': 526.4354, 'train_samples_per_second': 60.976, 'train_steps_per_second': 7.627, 'total_flos': 882342924149760.0, 'train_loss': 0.12738031746058784, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.20577672123908997,
 'eval_accuracy': 0.9317757009345794,
 'eval_runtime': 7.5473,
 'eval_samples_per_second': 283.547,
 'eval_steps_per_second': 35.51,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-bert-base-cased-punct/tree/main/'

#### Roberta-base

##### Nincs adat előfeldolgozás

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.335700,0.251103,0.945794
2,0.116200,0.160494,0.968224
3,0.061500,0.095634,0.980374
4,0.028500,0.191504,0.971963
5,0.009800,0.124148,0.982710


TrainOutput(global_step=4015, training_loss=0.09794530905133686, metrics={'train_runtime': 792.6059, 'train_samples_per_second': 40.499, 'train_steps_per_second': 5.066, 'total_flos': 1516931845241040.0, 'train_loss': 0.09794530905133686, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.09563366323709488,
 'eval_accuracy': 0.9803738317757009,
 'eval_runtime': 12.4584,
 'eval_samples_per_second': 171.771,
 'eval_steps_per_second': 21.512,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-roberta-base/tree/main/'

##### + Kisbetűsítés

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.370500,0.213027,0.961215
2,0.123000,0.139864,0.970561
3,0.095500,0.127316,0.977570
4,0.055400,0.168453,0.973832
5,0.029400,0.153439,0.974766


TrainOutput(global_step=4015, training_loss=0.1214937806546985, metrics={'train_runtime': 822.8566, 'train_samples_per_second': 39.01, 'train_steps_per_second': 4.879, 'total_flos': 1537493563106400.0, 'train_loss': 0.1214937806546985, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.1273161768913269,
 'eval_accuracy': 0.9775700934579439,
 'eval_runtime': 12.6647,
 'eval_samples_per_second': 168.974,
 'eval_steps_per_second': 21.161,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-roberta-base-lowercase/tree/main/'

##### + Stopwordök törlése

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.354000,0.267052,0.947664
2,0.152900,0.161685,0.956542
3,0.107100,0.136984,0.970561
4,0.048200,0.186082,0.971028
5,0.026300,0.186381,0.971028


TrainOutput(global_step=4015, training_loss=0.13253822295925352, metrics={'train_runtime': 735.2811, 'train_samples_per_second': 43.657, 'train_steps_per_second': 5.46, 'total_flos': 1310258111255760.0, 'train_loss': 0.13253822295925352, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.13698376715183258,
 'eval_accuracy': 0.9705607476635514,
 'eval_runtime': 10.8294,
 'eval_samples_per_second': 197.61,
 'eval_steps_per_second': 24.747,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-roberta-base-stopwords/tree/main/'

##### + URLek törlése

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.399000,0.274546,0.918224
2,0.263200,0.263427,0.936916
3,0.214200,0.210264,0.936916
4,0.140700,0.261238,0.951402
5,0.082600,0.217006,0.954206


TrainOutput(global_step=4015, training_loss=0.21002266647509887, metrics={'train_runtime': 651.614, 'train_samples_per_second': 49.262, 'train_steps_per_second': 6.162, 'total_flos': 1077495271687440.0, 'train_loss': 0.21002266647509887, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.21026372909545898,
 'eval_accuracy': 0.9369158878504673,
 'eval_runtime': 9.0388,
 'eval_samples_per_second': 236.757,
 'eval_steps_per_second': 29.65,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-roberta-base-url/tree/main/'

##### + HTML speciális karakterek törlése

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.472300,0.633783,0.618224
2,0.281900,0.248001,0.921495
3,0.256200,0.257468,0.921495
4,0.168600,0.222918,0.949533
5,0.097400,0.230581,0.950467


TrainOutput(global_step=4015, training_loss=0.25083392471037946, metrics={'train_runtime': 630.6817, 'train_samples_per_second': 50.897, 'train_steps_per_second': 6.366, 'total_flos': 1067353162725360.0, 'train_loss': 0.25083392471037946, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.22291846573352814,
 'eval_accuracy': 0.9495327102803738,
 'eval_runtime': 9.033,
 'eval_samples_per_second': 236.909,
 'eval_steps_per_second': 29.669,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-roberta-base-html/tree/main/'

##### + Emojik törlése

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.361800,0.297017,0.928505
2,0.169800,0.160019,0.952804
3,0.116500,0.186403,0.959813
4,0.063800,0.202877,0.960280
5,0.027700,0.213771,0.962617


TrainOutput(global_step=4015, training_loss=0.1409684130170407, metrics={'train_runtime': 602.7426, 'train_samples_per_second': 53.257, 'train_steps_per_second': 6.661, 'total_flos': 1025436282718320.0, 'train_loss': 0.1409684130170407, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.1600191444158554,
 'eval_accuracy': 0.952803738317757,
 'eval_runtime': 8.1705,
 'eval_samples_per_second': 261.918,
 'eval_steps_per_second': 32.801,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-roberta-base-emoji/tree/main/'

##### + Írásjelek törlése

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.509200,0.369274,0.885514
2,0.359300,0.381472,0.862150
3,0.350500,0.404878,0.827570
4,0.361000,0.417999,0.875701
5,0.256600,0.327477,0.894860


TrainOutput(global_step=4015, training_loss=0.37773219805725783, metrics={'train_runtime': 538.7414, 'train_samples_per_second': 59.583, 'train_steps_per_second': 7.453, 'total_flos': 841462043923200.0, 'train_loss': 0.37773219805725783, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3274768590927124,
 'eval_accuracy': 0.8948598130841121,
 'eval_runtime': 6.7877,
 'eval_samples_per_second': 315.276,
 'eval_steps_per_second': 39.483,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-roberta-base-punct/tree/main/'

#### Deberta-base

per_device_train_batch_size és per_device_eval_batch_size 4-re lett állítva cuda out of memory hibák miatt

##### Nincs adat előfeldolgozás

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.350900,0.742011,0.476636
2,0.224200,0.313964,0.940187
3,0.125100,0.233790,0.959346
4,0.126200,0.189836,0.971028
5,0.095600,0.157260,0.974766


TrainOutput(global_step=8025, training_loss=0.21517548932464695, metrics={'train_runtime': 1344.5607, 'train_samples_per_second': 23.874, 'train_steps_per_second': 5.968, 'total_flos': 1520863019227872.0, 'train_loss': 0.21517548932464695, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.15726041793823242,
 'eval_accuracy': 0.974766355140187,
 'eval_runtime': 18.2002,
 'eval_samples_per_second': 117.581,
 'eval_steps_per_second': 29.395,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-deberta-base/tree/main/'

##### + Kisbetűsítés

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.233900,0.230216,0.953271
2,0.169800,0.498100,0.911215
3,0.185800,0.243242,0.958879
4,0.132800,0.167346,0.967757
5,0.086800,0.227660,0.963551


TrainOutput(global_step=8025, training_loss=0.17147538347036295, metrics={'train_runtime': 1411.36, 'train_samples_per_second': 22.744, 'train_steps_per_second': 5.686, 'total_flos': 1538564239996032.0, 'train_loss': 0.17147538347036295, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.16734550893306732,
 'eval_accuracy': 0.9677570093457943,
 'eval_runtime': 19.3584,
 'eval_samples_per_second': 110.546,
 'eval_steps_per_second': 27.637,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-deberta-base-lowercase/tree/main/'

##### + Stopwordök törlése

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.334300,0.326213,0.919626
2,0.388900,0.315699,0.927570
3,0.232700,0.298298,0.938318
4,0.226100,0.212725,0.952804
5,0.162900,0.210204,0.961215


TrainOutput(global_step=8025, training_loss=0.2835314293293938, metrics={'train_runtime': 1372.1072, 'train_samples_per_second': 23.395, 'train_steps_per_second': 5.849, 'total_flos': 1260092179906080.0, 'train_loss': 0.2835314293293938, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.21020358800888062,
 'eval_accuracy': 0.961214953271028,
 'eval_runtime': 20.539,
 'eval_samples_per_second': 104.192,
 'eval_steps_per_second': 26.048,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-deberta-base-stopwords/tree/main/'

##### + URLek törlése

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.348500,0.375337,0.909813
2,0.308200,0.378332,0.906075
3,0.290200,0.346062,0.924299
4,0.325200,0.355584,0.917757
5,0.198500,0.291726,0.938318


TrainOutput(global_step=8025, training_loss=0.3175858378781708, metrics={'train_runtime': 1307.1634, 'train_samples_per_second': 24.557, 'train_steps_per_second': 6.139, 'total_flos': 1003970605741392.0, 'train_loss': 0.3175858378781708, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.2917255163192749,
 'eval_accuracy': 0.9383177570093458,
 'eval_runtime': 17.6326,
 'eval_samples_per_second': 121.366,
 'eval_steps_per_second': 30.342,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-deberta-base-url/tree/main/'

##### + HTML speciális karakterek törlése

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.403100,0.492666,0.880841
2,0.351300,0.375272,0.895794
3,0.508900,0.618101,0.811682
4,0.460000,0.408392,0.900467
5,0.308500,0.391730,0.907944


TrainOutput(global_step=8025, training_loss=0.42407046731015974, metrics={'train_runtime': 1323.6866, 'train_samples_per_second': 24.25, 'train_steps_per_second': 6.063, 'total_flos': 993917557897824.0, 'train_loss': 0.42407046731015974, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.37527167797088623,
 'eval_accuracy': 0.8957943925233645,
 'eval_runtime': 18.2272,
 'eval_samples_per_second': 117.407,
 'eval_steps_per_second': 29.352,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-deberta-base-html/tree/main/'

##### + Emojik törlése

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.323900,0.424786,0.909346
2,0.265300,0.296778,0.937850
3,0.235300,0.364263,0.930374
4,0.234100,0.416919,0.926636
5,0.136500,0.252702,0.946262


TrainOutput(global_step=8025, training_loss=0.2601341928499881, metrics={'train_runtime': 1287.8089, 'train_samples_per_second': 24.926, 'train_steps_per_second': 6.232, 'total_flos': 962292494595792.0, 'train_loss': 0.2601341928499881, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.2527017295360565,
 'eval_accuracy': 0.9462616822429907,
 'eval_runtime': 22.8428,
 'eval_samples_per_second': 93.684,
 'eval_steps_per_second': 23.421,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-deberta-base-emoji/tree/main/'

##### + Írásjelek törlése

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.540000,0.468130,0.837383
2,0.700200,0.711336,0.523364
3,0.696700,0.693828,0.523364
4,0.696100,0.695935,0.476636
5,0.693500,0.692092,0.523364


TrainOutput(global_step=8025, training_loss=0.658491289132852, metrics={'train_runtime': 1278.7235, 'train_samples_per_second': 25.103, 'train_steps_per_second': 6.276, 'total_flos': 798145409613216.0, 'train_loss': 0.658491289132852, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.46812960505485535,
 'eval_accuracy': 0.8373831775700935,
 'eval_runtime': 17.7623,
 'eval_samples_per_second': 120.48,
 'eval_steps_per_second': 30.12,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-deberta-base-punct/tree/main/'

#### Deberta-base - gradiens akkumuláció

per_device_train_batch_size és per_device_eval_batch_size 4-re + gradient_accumulation_steps 2-re lett állítva cuda out of memory hibák miatt

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
0,0.238100,0.164861,0.964953
2,0.044100,0.143205,0.975234
4,0.005500,0.159495,0.978972


TrainOutput(global_step=4010, training_loss=0.07159855305551792, metrics={'train_runtime': 1150.538, 'train_samples_per_second': 27.9, 'train_steps_per_second': 3.485, 'total_flos': 1520010294655008.0, 'train_loss': 0.07159855305551792, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.14320535957813263,
 'eval_accuracy': 0.9752336448598131,
 'eval_runtime': 17.3232,
 'eval_samples_per_second': 123.534,
 'eval_steps_per_second': 30.883,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1699276621.2c5b2a8115d2.330.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1699275433.2c5b2a8115d2.330.0:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

'https://huggingface.co/Denyol/FakeNews-deberta-base-grad/tree/main/'

#### Bert-large-cased

per_device_train_batch_size és per_device_eval_batch_size 4-re lett állítva cuda out of memory hibák miatt

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.708000,0.698160,0.523364
2,0.710700,0.701676,0.523364
3,0.703100,0.693221,0.523364
4,0.704800,0.696438,0.476636
5,0.700100,0.692061,0.523364


TrainOutput(global_step=8025, training_loss=0.7052062313281858, metrics={'train_runtime': 3031.8411, 'train_samples_per_second': 10.588, 'train_steps_per_second': 2.647, 'total_flos': 5015851176703248.0, 'train_loss': 0.7052062313281858, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6920610070228577,
 'eval_accuracy': 0.5233644859813084,
 'eval_runtime': 37.6312,
 'eval_samples_per_second': 56.868,
 'eval_steps_per_second': 14.217,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-bert-large-cased/tree/main/'

#### Bert-large-cased - gradiens akkumuláció

per_device_train_batch_size és per_device_eval_batch_size 4-re + gradient_accumulation_steps 2-re lett állítva cuda out of memory hibák miatt

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
0,0.326600,0.164789,0.962150
2,0.084400,0.123696,0.974299
4,0.013700,0.147294,0.974766


TrainOutput(global_step=4010, training_loss=0.1049170059750572, metrics={'train_runtime': 2596.9901, 'train_samples_per_second': 12.36, 'train_steps_per_second': 1.544, 'total_flos': 5013142751178576.0, 'train_loss': 0.1049170059750572, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.12369615584611893,
 'eval_accuracy': 0.9742990654205608,
 'eval_runtime': 39.6198,
 'eval_samples_per_second': 54.013,
 'eval_steps_per_second': 13.503,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1699280186.27228d013e56.620.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

events.out.tfevents.1699277510.27228d013e56.620.0:   0%|          | 0.00/7.76k [00:00<?, ?B/s]

'https://huggingface.co/Denyol/FakeNews-bert-large-cased-grad/tree/main/'

#### Bert-large-cased - stabilizálás

per_device_train_batch_size és per_device_eval_batch_size 4-re + gradient_accumulation_steps 2-re lett állítva cuda out of memory hibák miatt

Stabilizálás céljából az alábbi paraméterek lettek beállítva:

learning_rate=3e-5,
warmup_ratio=0.1,
adam_epsilon=1e-6,
adam_beta1=0.9,
adam_beta2=0.999,
weight_decay=0.01,
max_grad_norm=1.0

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
0,0.354900,0.325529,0.945327
2,0.041200,0.101995,0.982710
4,0.000100,0.131454,0.982710


TrainOutput(global_step=4010, training_loss=0.08883251410552928, metrics={'train_runtime': 2802.5035, 'train_samples_per_second': 11.454, 'train_steps_per_second': 1.431, 'total_flos': 5013142751178576.0, 'train_loss': 0.08883251410552928, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.10199499130249023,
 'eval_accuracy': 0.9827102803738318,
 'eval_runtime': 46.526,
 'eval_samples_per_second': 45.996,
 'eval_steps_per_second': 11.499,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1699813661.e84210fa9b57.2143.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

'https://huggingface.co/Denyol/FakeNews-bert-large-cased-stable/tree/main/'

#### Roberta-large

per_device_train_batch_size és per_device_eval_batch_size 4-re lett állítva cuda out of memory hibák miatt

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.714200,0.695417,0.523364
2,0.709700,0.694672,0.476636
3,0.703300,0.749915,0.476636
4,0.691000,1.226814,0.476636
5,0.669300,1.570428,0.476636


TrainOutput(global_step=8025, training_loss=0.6985662366445191, metrics={'train_runtime': 2930.603, 'train_samples_per_second': 10.953, 'train_steps_per_second': 2.738, 'total_flos': 4622801843505888.0, 'train_loss': 0.6985662366445191, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6946719884872437,
 'eval_accuracy': 0.4766355140186916,
 'eval_runtime': 36.7509,
 'eval_samples_per_second': 58.23,
 'eval_steps_per_second': 14.557,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-roberta-large/tree/main/'

#### Roberta-large - gradiens akkumuláció

per_device_train_batch_size és per_device_eval_batch_size 4-re + gradient_accumulation_steps 2-re lett állítva cuda out of memory hibák miatt

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
0,0.391700,0.703146,0.523364
2,0.701100,0.692059,0.523364
4,0.686500,0.712464,0.476636


TrainOutput(global_step=4010, training_loss=0.6615772537459756, metrics={'train_runtime': 2576.5131, 'train_samples_per_second': 12.459, 'train_steps_per_second': 1.556, 'total_flos': 4620209909401632.0, 'train_loss': 0.6615772537459756, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6920585632324219,
 'eval_accuracy': 0.5233644859813084,
 'eval_runtime': 40.8947,
 'eval_samples_per_second': 52.329,
 'eval_steps_per_second': 13.082,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1699364775.5e4b9ad8fe94.1054.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

events.out.tfevents.1699362002.5e4b9ad8fe94.1054.0:   0%|          | 0.00/7.57k [00:00<?, ?B/s]

'https://huggingface.co/Denyol/FakeNews-roberta-large-grad/tree/main/'

#### Roberta-large - stabilizálás

per_device_train_batch_size és per_device_eval_batch_size 4-re + gradient_accumulation_steps 2-re lett állítva cuda out of memory hibák miatt

Stabilizálás céljából az alábbi paraméterek lettek beállítva:

learning_rate=3e-5,
warmup_ratio=0.1,
adam_epsilon=1e-6,
adam_beta1=0.9,
adam_beta2=0.98,
weight_decay=0.1

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
0,0.430700,0.326190,0.935047
2,0.274800,0.206588,0.959346
4,0.117000,0.175651,0.966822


TrainOutput(global_step=4010, training_loss=0.24454478071514804, metrics={'train_runtime': 2546.526, 'train_samples_per_second': 12.605, 'train_steps_per_second': 1.575, 'total_flos': 4620209909401632.0, 'train_loss': 0.24454478071514804, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.17565101385116577,
 'eval_accuracy': 0.9668224299065421,
 'eval_runtime': 37.3438,
 'eval_samples_per_second': 57.305,
 'eval_steps_per_second': 14.326,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1699867794.39facd9adc8d.396.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

'https://huggingface.co/Denyol/FakeNews-roberta-large-stable/tree/main/'

#### Deberta-large

per_device_train_batch_size és per_device_eval_batch_size 2-re lett állítva cuda out of memory hibák miatt

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.696000,0.692826,0.523364
2,0.706300,0.692203,0.523364
3,0.697800,0.692892,0.523364
4,0.694700,0.695788,0.476636
5,0.612200,0.509252,0.797196


TrainOutput(global_step=16050, training_loss=0.6991121635555849, metrics={'train_runtime': 5735.7788, 'train_samples_per_second': 5.596, 'train_steps_per_second': 2.798, 'total_flos': 4443035284925160.0, 'train_loss': 0.6991121635555849, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.509251594543457,
 'eval_accuracy': 0.797196261682243,
 'eval_runtime': 63.1815,
 'eval_samples_per_second': 33.871,
 'eval_steps_per_second': 16.935,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Denyol/FakeNews-deberta-large/tree/main/'

#### Deberta-large - gradiens akkumuláció

per_device_train_batch_size és per_device_eval_batch_size 4-re + gradient_accumulation_steps 2-re + gradient_checkpointing True-ra lett állítva cuda out of memory hibák miatt

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.525500,0.415917,0.854673
2,0.484900,0.466951,0.771028
4,0.558100,0.754828,0.476636


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=4010, training_loss=0.5074189521428059, metrics={'train_runtime': 4053.6968, 'train_samples_per_second': 7.919, 'train_steps_per_second': 0.989, 'total_flos': 5386759297828896.0, 'train_loss': 0.5074189521428059, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4159168303012848,
 'eval_accuracy': 0.8546728971962617,
 'eval_runtime': 52.5075,
 'eval_samples_per_second': 40.756,
 'eval_steps_per_second': 10.189,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1699367045.3f763f6117d2.298.0:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

events.out.tfevents.1699371226.3f763f6117d2.298.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/Denyol/FakeNews-deberta-large-grad/tree/main/'

#### Deberta-large - stabilizálás

per_device_train_batch_size és per_device_eval_batch_size 4-re + gradient_accumulation_steps 2-re + gradient_checkpointing True-ra lett állítva cuda out of memory hibák miatt

Stabilizálás céljából az alábbi paraméterek lettek beállítva:

learning_rate=3e-5,
warmup_ratio=0.1,
adam_epsilon=1e-6,
adam_beta1=0.9,
adam_beta2=0.999,
weight_decay=0.01,
max_grad_norm=1.0

In [ ]:
trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.337300,0.163463,0.968224
2,0.041900,0.114132,0.978505
4,0.002600,0.168044,0.980841


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=4010, training_loss=0.08329170476861514, metrics={'train_runtime': 4077.867, 'train_samples_per_second': 7.872, 'train_steps_per_second': 0.983, 'total_flos': 5386759297828896.0, 'train_loss': 0.08329170476861514, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.10483434796333313,
 'eval_accuracy': 0.9771028037383177,
 'eval_runtime': 47.2878,
 'eval_samples_per_second': 45.255,
 'eval_steps_per_second': 11.314,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1699869138.c7cf6f92677a.568.0:   0%|          | 0.00/7.63k [00:00<?, ?B/s]

events.out.tfevents.1699873424.c7cf6f92677a.568.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

'https://huggingface.co/Denyol/FakeNews-deberta-large-stable/tree/main/'

## AutoPrompt

### Környezet beállítása

In [ ]:
# Git repo klónozása
! git clone https://github.com/ucinlp/autoprompt.git

Cloning into 'autoprompt'...
remote: Enumerating objects: 4645, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 4645 (delta 10), reused 11 (delta 5), pack-reused 4593
Receiving objects: 100% (4645/4645), 78.06 MiB | 17.88 MiB/s, done.
Resolving deltas: 100% (3454/3454), done.


In [ ]:
# PyTorch telepítés
! pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 764.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu121
    Uninstalling torchvision-0.16.0+cu121:
      Successfully uninstalled torchvision-0.16.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.1.0+cu121
    Uninstalling torchaudio-2.1.0+cu121:
      Successfully uninstalled torchaudio-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [ ]:
import torch
print(torch.__version__)

import torchvision
print(torchvision.__version__)

import torchaudio
print(torchaudio.__version__)

2.1.2+cu121
0.16.2+cu121
2.1.2+cu121


In [ ]:
%cd autoprompt/
! ls

/content/autoprompt
app	    index.html	prompts     README.md	      scripts	tests
autoprompt  LICENSE	pytest.ini  requirements.txt  setup.py


requirements.txt és autoprompt/create_trigger.py tartalmának szerkesztése:

\
**requirements.txt:**

streamlit==1.28.2\
tqdm==4.65.0\
pandas==2.1.3\
numpy==1.26.2\
transformers==4.35.2\
spacy==3.7.2\
termcolor==2.3.0\
colorama==0.4.6\
matplotlib==3.8.2\
pytest==7.4.3

\
**create_trigger.py:**

52\. sor átírása erre: \
logits = self._model(**model_inputs).logits


In [ ]:
# Függőségek telepítése
! pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.1/325.1 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
    

In [ ]:
# Restart után:
%cd autoprompt/
! ls

/content/autoprompt
app	    index.html	prompts     README.md	      scripts	tests
autoprompt  LICENSE	pytest.ini  requirements.txt  setup.py


In [ ]:
# Spacy telepítés
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Dataset átalakítás "Autoprompt kompatibilissé"

In [ ]:
# Dataset repojának klónozása
! git clone https://huggingface.co/datasets/nanyy1025/covid_fake_news

Cloning into 'covid_fake_news'...
remote: Enumerating objects: 25, done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 25
Unpacking objects: 100% (25/25), 824.78 KiB | 3.11 MiB/s, done.


In [ ]:
# Datasetek betöltése, importok
import pandas as pd
from transformers import AutoTokenizer

df1 = pd.read_csv('covid_fake_news/train.csv')
df2 = pd.read_csv('covid_fake_news/val.csv')
df3 = pd.read_csv('covid_fake_news/test.csv')

In [ ]:
# Datasetek formázása
df1.rename(columns={'tweet' : 'sentence'}, inplace = True)
df1.drop(columns=['id'], axis=1, inplace=True)

df2.rename(columns={'tweet' : 'sentence'}, inplace = True)
df2.drop(columns=['id'], axis=1, inplace=True)

df3.rename(columns={'tweet' : 'sentence'}, inplace = True)
df3.drop(columns=['id'], axis=1, inplace=True)

In [ ]:
# Szekvencia hosszak printelése
print(df1.sentence.str.len().sort_values())
print(df2.sentence.str.len().sort_values())
print(df3.sentence.str.len().sort_values())

1140      18
268       20
1883      21
3141      22
2407      23
        ... 
4349     880
773      922
6306    2086
1109    5040
270     8846
Name: sentence, Length: 6420, dtype: int64
1139      26
2032      29
528       30
856       32
1408      33
        ... 
204      520
242      520
452      633
47       723
1733    2135
Name: sentence, Length: 2140, dtype: int64
1784       25
265        28
933        30
2134       31
287        33
        ...  
364       673
1141      895
535      1301
506      2160
1469    10170
Name: sentence, Length: 2140, dtype: int64


In [ ]:
# Index kreálás, numpy-ra konvertálás, tokenizálás
idx1 = [i for i in range(len(df1.index))]
x_train = df1['sentence'].values[idx1]

idx2 = [i for i in range(len(df2.index))]
x_val = df2['sentence'].values[idx2]

idx3 = [i for i in range(len(df3.index))]
x_test = df3['sentence'].values[idx3]

model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_tokenized = tokenizer(list(x_train), return_tensors='pt', truncation=True, padding=True)
val_tokenized = tokenizer(list(x_val), return_tensors='pt', truncation=True, padding=True)
test_tokenized = tokenizer(list(x_test), return_tensors='pt', truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# Token check
train_tokenized['input_ids'][6306]

tensor([  101,  2268,  3891, 10128,   151,   119,   162,   119,  1552,  1196,
         5694,  1121,  1884, 15789, 27608,   119,   790,   138,  1888,   113,
         9273,   114,  1104,  1103,  3355, 19875,   140,  1233,  7326,  4578,
         1116,  7447,  8764,   119,  1135,  2196,  1126,  3427,  2613,  1395,
          119,  1109,  1825,  7973,  1867,  1115,  1103,  1433,  1110,  4009,
         1165,  1122,  1867,  1175,  1132, 18732, 23314,  2137,   118,  1627,
         4420,  1107,  1103,  2704,   119,  1646,  1352,   118,  2205,  2394,
          118,  1826, 11482, 27453,  3052,  1794,  1110,  1208,  1714,  1111,
         2490,  1106,  8343,  1234,  1106,  2215,  1313,   119,  1955,  1884,
        15789, 27608,  4420,  1138,  6203,  1107,  9446,   119,   140, 20737,
         4206,  1115,  5464, 19040, 24861,  1447, 11200,  1116,  2281,  1884,
        15789, 27608,   119,  1109,  1703,  1295,  1104, 18732, 23314,  2137,
          118,  1627,  6209, 10558,  1107,  2268, 25134,  3524, 

In [ ]:
# Túl hosszú szekvenciák szűrése, ellenőrzés, új dataset mentése
limit = 2100
df1 = df1[df1.sentence.str.len() <= limit]
df2 = df2[df2.sentence.str.len() <= limit]
df3 = df3[df3.sentence.str.len() <= limit]

print(df1.sentence.str.len().sort_values())
print(df2.sentence.str.len().sort_values())
print(df3.sentence.str.len().sort_values())

df1.to_csv('covid_fake_news/train.tsv', sep='\t', index=False)
df2.to_csv('covid_fake_news/val.tsv', sep='\t', index=False)
df3.to_csv('covid_fake_news/test.tsv', sep='\t', index=False)

1140      18
268       20
1883      21
3141      22
2407      23
        ... 
3082     581
4147     695
4349     880
773      922
6306    2086
Name: sentence, Length: 6418, dtype: int64
1139     26
2032     29
528      30
856      32
1408     33
       ... 
1379    465
242     520
204     520
452     633
47      723
Name: sentence, Length: 2139, dtype: int64
1784      25
265       28
933       30
2134      31
287       33
        ... 
31       450
1318     470
364      673
1141     895
535     1301
Name: sentence, Length: 2138, dtype: int64


### Futtatás

#### Bert-base-cased

##### 1\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 570/570 [00:00<00:00, 2.39MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 436M/436M [00:02<00:00, 156MB/s]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expec

##### 2\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly id

##### 3\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 24

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 24 \
    --eval-size 24 \
    --iters 5 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly id

##### 4\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 10 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 436M/436M [00:06<00:00, 70.5MB/s]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from 

##### 5\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 10 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly id

##### 6\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 24

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 24 \
    --eval-size 24 \
    --iters 10 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly id

##### 7\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly id

##### 8\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly id

##### 9\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 24

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 24 \
    --eval-size 24 \
    --iters 5 \
    --model-name bert-base-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly id

#### Roberta-base

##### 1\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 268/268 [00:18<00:00, 14.47it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main_

##### 2\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 134/134 [00:20<00:00,  6.68it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main_

##### 3\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 24

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 24 \
    --eval-size 24 \
    --iters 5 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 90/90 [00:21<00:00,  4.28it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main__:

##### 4\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 10 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 268/268 [00:19<00:00, 13.89it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main_

##### 5\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 10 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 134/134 [00:19<00:00,  7.04it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main_

##### 6\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 24

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 24 \
    --eval-size 24 \
    --iters 10 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 90/90 [00:20<00:00,  4.39it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main__:

##### 7\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 268/268 [00:19<00:00, 13.84it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main_

##### 8\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 134/134 [00:19<00:00,  6.77it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main_

##### 9\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 24

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 24 \
    --eval-size 24 \
    --iters 5 \
    --model-name roberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 90/90 [00:20<00:00,  4.33it/s]
INFO:__main__:Dev metric: 0.6100981767180925
INFO:__main__:

#### Deberta-base

##### 1\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

##### 2\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

##### 3\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 14 (CUDA out of memory 18, 20, 22 és 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 14 \
    --eval-size 14 \
    --iters 5 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

##### 4\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 10 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

##### 5\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 12 (CUDA out of memory 16-nál)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 12 \
    --eval-size 12 \
    --iters 10 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

##### 6\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 10 (CUDA out of memory 14, 18, 20, 22 és 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 10 \
    --eval-size 10 \
    --iters 10 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

##### 7\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

##### 8\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 12 (CUDA of out memory 16-nál)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 12 \
    --eval-size 12 \
    --iters 5 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

##### 9\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 10 (CUDA out of memory 14, 18, 20, 22 és 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 10 \
    --eval-size 10 \
    --iters 5 \
    --model-name microsoft/deberta-base

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests

#### Bert-large-cased

##### 1\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 762/762 [00:00<00:00, 3.43MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.34G/1.34G [01:44<00:00, 12.8MB/s]
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT e

##### 2\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

##### 3\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 24

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 24 \
    --eval-size 24 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

##### 4\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 10 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

##### 5\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 10 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

##### 6\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 24

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 24 \
    --eval-size 24 \
    --iters 10 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 762/762 [00:00<00:00, 3.09MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.34G/1.34G [00:08<00:00, 152MB/s]
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT ex

##### 7\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

##### 8\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 762/762 [00:00<00:00, 4.04MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.34G/1.34G [00:07<00:00, 181MB/s]
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT ex

##### 9\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 14 (CUDA ouf of memory 24, 22 és 20 és 18-nál)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 14 \
    --eval-size 14 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

#### Roberta-large

##### 1\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 482/482 [00:00<00:00, 2.38MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.42G/1.42G [00:08<00:00, 173MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 1.26MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 1.84MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:01<00:00, 1.11MB/s]
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor

##### 2\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 134/134 [00:52<00:00,  2.55it/s]
INFO:__main__:Dev metric: 0.603085553997195
INFO:__main__

##### 3\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 22 (CUDA out of memory 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 22 \
    --eval-size 22 \
    --iters 5 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 482/482 [00:00<00:00, 1.79MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.42G/1.42G [00:07<00:00, 194MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 5.41MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 34.4MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 16.4MB/s]
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor

##### 4\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 10 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 482/482 [00:00<00:00, 1.91MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.42G/1.42G [00:11<00:00, 129MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 917kB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 25.2MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:01<00:00, 960kB/s]
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([

##### 5\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 16 \
    --eval-size 16 \
    --iters 10 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 134/134 [00:48<00:00,  2.75it/s]
INFO:__main__:Dev metric: 0.603085553997195
INFO:__main__

##### 6\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 22 (CUDA out of memory 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 22 \
    --eval-size 22 \
    --iters 10 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 98/98 [00:44<00:00,  2.20it/s]
INFO:__main__:Dev metric: 0.603085553997195
INFO:__main__:I

##### 7\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor([[   3,    3,    3,    3, 9091]], device='cuda:0'), tensor([[47173, 38547, 33959, 13685, 28162]], device='cuda:0')]
INFO:__main__:Loading datasets
INFO:__main__:Evaluating
100% 268/268 [00:47<00:00,  5.69it/s]
INFO:__main__:Dev metric: 0.603085553997195
INFO:__main__

##### 8\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 16

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 16 \
    --eval-size 16 \
    --iters 5 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 482/482 [00:00<00:00, 2.17MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.42G/1.42G [00:08<00:00, 163MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 15.6MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 18.5MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 47.3MB/s]
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor

##### 9\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 20 (CUDA out of memory 22 és 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 20 \
    --eval-size 20 \
    --iters 5 \
    --model-name roberta-large

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 482/482 [00:00<00:00, 2.11MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.42G/1.42G [00:08<00:00, 169MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 1.27MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 11.2MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 46.6MB/s]
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:{'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['tests', 'results', 'testing', 'reports', 'cases']}
INFO:__main__:[tensor

#### Deberta-large

##### 1\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['test

##### 2\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 12 (CUDA out of memory 16 és 14-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 12 \
    --eval-size 12 \
    --iters 5 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['test

##### 3\. Kisérlet, 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 10 (CUDA out of memory 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 10 \
    --eval-size 10 \
    --iters 5 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['test

##### 4\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 10 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['test

##### 5\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 12 (CUDA out of memory 16 és 14-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 12 \
    --eval-size 12 \
    --iters 10 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['test

##### 6\. Kisérlet, 10 iteráció, accumulation-steps: 10 (default), batch és eval méret: 10 (CUDA out of memory 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 10 \
    --eval-size 10 \
    --iters 10 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 475/475 [00:00<00:00, 2.30MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
pytorch_model.bin: 100% 1.63G/1.63G [00:15<00:00, 104MB/s]
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

##### 7\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 8

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['test

##### 8\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 12 (CUDA out of memory 16 és 14-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 12 \
    --eval-size 12 \
    --iters 5 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Label map: {'fake': ['vaccine', 'masks', 'brainwash', 'man-made', 'force'], 'real': ['test

##### 9\. Kisérlet, 5 iteráció, accumulation-steps: 20, batch és eval méret: 10 (CUDA out of memory 24-nél)

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '<s> {sentence} [T] [T] [T] [P] . </s>' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --accumulation-steps 20 \
    --bsz 10 \
    --eval-size 10 \
    --iters 5 \
    --model-name microsoft/deberta-large

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/content/autoprompt/autoprompt/create_trigger.py", line 523, in <module>
    run_model(args)
  File "/content/autoprompt/autoprompt/create_trigger.py", line 200, in run_model
    config, model, tokenizer = load_pretrained(args.model_name)
  File "/content/autoprompt/autoprompt/create_trigger.py", line 112, in load_pretrained
  

#### Legjobb modell: Bert-large-cased
Paraméterek: 5 iteráció, accumulation-steps: 10 (default), batch és eval méret: 8

##### Újra futtatás kétszer ugyanazokkal a paraméterekkel:

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --seed 42 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 762/762 [00:00<00:00, 3.61MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.34G/1.34G [00:05<00:00, 253MB/s]
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT ex

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["vaccine", "masks", "brainwash", "man-made", "force"], "real": ["tests", "results", "testing", "reports", "cases"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --seed 18 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

##### Újra futtatás más label mapekkel:

###### 1. variáció

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["inaccurate", "lockdown", "graves", "infestation", "older"], "real": ["update", "overview", "antibodies", "recoveries", "help"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

###### 2. variáció

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["5G", "won’t transmit", "don’t work", "harmful", "catch"], "real": ["conspiracy", "symptoms", "protect", "professionals", "death rate"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
config.json: 100% 762/762 [00:00<00:00, 4.05MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
model.safetensors: 100% 1.34G/1.34G [00:07<00:00, 188MB/s]
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT ex

###### 3. variáció

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["slammed", "causing", "forced", "Trump", "grounded"], "real": ["outcome", "importance", "active cases", "reported", "develop"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i

###### 4. variáció

In [ ]:
# Autoprompt futtatása
! python -m autoprompt.create_trigger \
    --train covid_fake_news/train.tsv \
    --dev covid_fake_news/val.tsv \
    --template '[CLS] {sentence} [T] [T] [T] [P] . [SEP]' \
    --label-map '{"fake": ["survive", "China", "packed", "sanitizer", "cone"], "real": ["samples", "response", "data", "care", "solidarity"]}' \
    --num-cand 100 \
    --bsz 8 \
    --eval-size 8 \
    --iters 5 \
    --model-name bert-large-cased

INFO:__main__:Loading model, tokenizer, etc.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly i